In [1]:
import pandas as pd
import numpy as np
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

# Plotly Express


In [2]:
import plotly.express as px

In [4]:
df_s = pd.read_csv('南棟客梯_inner.csv',index_col = 0)  
print(df_s.head())
print(df_s.tail())

   hour floor     week  count
1     0    01  weekday    706
2     1    01  weekday    434
3     2    01  weekday    196
4     3    01  weekday    148
5     4    01  weekday    137
     hour floor     week  count
860    19    B4  weekend     22
861    20    B4  weekend     22
862    21    B4  weekend     10
863    22    B4  weekend      3
864    23    B4  weekend      8


plotly express 有點像是 ggplot in R，<br>
只要指定x,y,color等就能自動幫你分類，<br>
下面的圖表以南棟電梯的外部電梯呼叫資料為例，<br>
x軸為樓層、以方向分開並上色，最後以時間(小時)作為動畫影格。

In [10]:
floor_list = ['B4','B3', 'B2','B1','01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15']

fig = px.bar(data_frame=df_s, 
             
             ## 這裡的data frame都先用 R aggregate過了，以節省運算時間
             x = "floor", y = "count",
             
             ## 以方向分兩張圖，並上色
#              facet_col="direction",color="direction",
             
             ## 自動生成slider
             animation_frame="hour",
             
             labels={"count":"Frequency",
                     "hour":"Local Time (GMT+8)"})

# X axis
fig.update_xaxes(nticks=len(floor_list), ##指定breaks的數量
                 
                 ## 手動將X的類別排序
                 type="category",
                 categoryarray=floor_list)

# Y axis
fig.update_yaxes(range=[0, 2000])

# Layout
fig.update_layout(title_text='Frequency of Elevator Calls by Floor')

fig.show()

In [11]:
fig = px.bar(data_frame=df_s, 
             x = "hour", y = "count",
#              facet_col="direction",color="direction",
             animation_frame="floor",
             labels={"count":"Frequency",
                     "hour":"Local Time (GMT+8)"})
# X axis
fig.update_xaxes(nticks=len(floor_list),
                 type="category",
                 categoryarray=[0,24])

# Y axis
fig.update_yaxes(range=[0, 2000])

# Layout
fig.update_layout(title_text='Frequency of Elevator Calls by Time')

fig.show()

## Data

In [13]:
import plotly.graph_objects as go
from ipywidgets import widgets

將不同大樓資料混在一起，依大樓名稱貼標

In [15]:
## 檔案名(註:資料是arregate後的table)

files_list = ['研究大樓_inner','北棟病床梯_inner','南棟客梯_inner', '南棟病床梯_inner', '北棟客梯_inner']

df = pd.DataFrame()

for i in files_list:
    data = pd.read_csv(i+'.csv', index_col = 0)
    data['building'] = i
    print(i)
    print(data.shape)
    
    df = df.append(data, ignore_index = True) 

## 請檢查表格的數量是否正確 有些表格有缺失(e.g.北棟客梯)
    
print(df.shape)
print(df.head())

研究大樓_inner
(864, 5)
北棟病床梯_inner
(864, 5)
南棟客梯_inner
(864, 5)
南棟病床梯_inner
(864, 5)
北棟客梯_inner
(912, 5)
(4368, 5)
   hour floor     week  count    building
0     0    01  weekday     45  研究大樓_inner
1     1    01  weekday     19  研究大樓_inner
2     2    01  weekday      9  研究大樓_inner
3     3    01  weekday     12  研究大樓_inner
4     4    01  weekday      9  研究大樓_inner


## Dash

In [17]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

app = dash.Dash()
app.layout = html.Div([
    html.H1('Outer calls'),
    html.H3('By Floor'),
    dcc.Dropdown(
        id="building1",
        options=[
            {'label': '研究大樓', 'value': '研究大樓'},
            {'label': '北棟病床梯', 'value': '北棟病床梯'},
            {'label': '南棟電梯', 'value': '南棟電梯'}
        ],
        value='研究大樓'
    ),
    dcc.Dropdown(
        id="week1",
        options=[
            {'label': '全日', 'value': 'total'},
            {'label': '平日', 'value': 'weekday'},
            {'label': '假日', 'value': 'weekend'}
        ],
        value='total'
    ),
    dcc.Graph(id="graph1"),
    
    
    html.H3('By Hour'),
    dcc.Dropdown(
        id="building2",
        options=[
            {'label': '研究大樓', 'value': '研究大樓'},
            {'label': '北棟病床梯', 'value': '北棟病床梯'},
            {'label': '北棟病床梯', 'value': '北棟病床梯'},
            {'label': '南棟電梯', 'value': '南棟電梯'},
            {'label': '南棟病床梯', 'value': '南棟病床梯'}
        ],
        value='研究大樓'
    ),
        dcc.Dropdown(
        id="week2",
        options=[
            {'label': '全日', 'value': 'total'},
            {'label': '平日', 'value': 'weekday'},
            {'label': '假日', 'value': 'weekend'}
        ],
        value='total'
    ),
    dcc.Graph(id="graph2")
])

@app.callback(
    Output('graph1', 'figure'),
    [Input('building1', 'value'),
     Input('week1', 'value')])
def update_figure(selected_building, selected_week):
    
    ## Building
    temp = df[df.building == selected_building]
    
    ## Weekday / Weekend
    if (selected_week != 'total'):
        temp = temp[temp.week == selected_week]
        
        
    fig = px.bar(data_frame=temp,
                 x = "floor", y = "count",
#                  facet_col="direction",color="direction",
                 animation_frame="hour",
                 labels={"count":"Frequency",
                         "hour":"Local Time (GMT+8)"})
    fig.update_xaxes(nticks=len(floor_list),
                     type="category",
                    categoryarray=floor_list)
    fig.update_yaxes(range=[0, 1000])
    fig.update_layout(title_text='Frequency of Elevator Calls in ' + selected_building + ' ('+ selected_week +')')
    
    return fig



@app.callback(
    Output('graph2', 'figure'),
    [Input('building2', 'value'),
     Input('week2', 'value')])
def update_figure(selected_building, selected_week):
    
    ## Building
    temp = df[df.building == selected_building]
    
    ## Weekday / Weekend
    if (selected_week != 'total'):
        temp = temp[temp.week == selected_week]
        
    
    fig = px.bar(data_frame=temp, 
             x = "hour", y = "count",
#              facet_col="direction",color="direction",
             animation_frame="floor",
             labels={"count":"Frequency",
                     "hour":"Local Time (GMT+8)"})
    fig.update_xaxes(nticks=len(floor_list),
                     type="category",
                     categoryarray=[0,24])
    fig.update_yaxes(range=[0, 1000])
    fig.update_layout(title_text='Frequency of Elevator Calls by Time in ' + selected_building + ' ('+ selected_week +')')
    return fig
        
app.run_server(debug=True, use_reloader=False)

Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Debugger PIN: 194-608-314
Debugger PIN: 194-608-314
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


OSError: [Errno 48] Address already in use